In [2]:
import pandas as pd
import numpy as np
from ipysheet import sheet, cell, column
import ipywidgets as widgets
import requests
import math
from PIL import Image
import io
import skimage

In [48]:
df = pd.read_csv('data_to_graph.csv', index_col = 0)

In [49]:
df

,occurences,title,poster_url,up_down,production_co
imdb_id,,,,,
tt0944947,2930259,Game of Thrones,https://m.media-amazon.com/images/M/MV5BYTRiND...,1,CBS
tt5753856,2843361,Dark,https://m.media-amazon.com/images/M/MV5BZjFlZj...,1,Netflix
tt2661044,2307627,The 100,https://m.media-amazon.com/images/M/MV5BNjRiYT...,-1,DC
tt7556122,2095728,The Old Guard,https://m.media-amazon.com/images/M/MV5BZTY5YT...,0,Netflix
tt6048922,1882505,Greyhound,https://m.media-amazon.com/images/M/MV5BZTFkZj...,0,Apple TV
tt1520211,1711230,The Walking Dead,https://m.media-amazon.com/images/M/MV5BYTUwOT...,1,Disney
tt2364582,1660859,Agents of S.H.I.E.L.D.,https://m.media-amazon.com/images/M/MV5BOTJhMj...,-1,Hulu
tt6468322,1623676,Money Heist,https://m.media-amazon.com/images/M/MV5BZDcxOG...,-1,Netflix
tt1502397,1598424,Bad Boys for Life,https://m.media-amazon.com/images/M/MV5BMWU0MG...,1,Peacock


In [ ]:
sheet1 = sheet(rows= df.shape[0], columns=2, row_headers=False, column_headers=False)

In [ ]:
column1_data = []

for i, (title, url) in enumerate(zip(df["title"], df["poster_url"])):
    res = requests.get(url=url)
    label1 = widgets.HTML(value="<h2>%2.2d</h2>"%(i+1), layout=widgets.Layout(margin="20px"))
    label2 = widgets.HTML(value="<h2>%s</h2>"%title, layout=widgets.Layout(margin="20px"))
    image = widgets.Image(value = res.content, width=75)
    hbox = widgets.HBox([label1, image, label2])
    hbox.layout.align_items = "center"
    hbox.layout.width="600px"
    column1_data.append(hbox)

In [ ]:
column1 = column(0, column1_data)

In [ ]:
column2_data = []

max_value = math.ceil(df.occurences.max() / 1e6 + 2)

for occurence in df.occurences:
        occurence = occurence / 1e6
        label1 = widgets.HTML("<h3>%.2f M</h3>"%occurence, layout=widgets.Layout(margin="20px"))
        progress = widgets.FloatProgress(value=occurence, 
                                         min=0, max=max_value,
                                         layout=widgets.Layout(width="400px", height="80px"))
        hbox = widgets.HBox([label1, progress])
        hbox.layout.align_items = "center"
        hbox.layout.height="100%"
        hbox.layout.width="100%"
        column2_data.append(hbox)

In [ ]:
column2 = column(1, column2_data)

In [ ]:
sheet1

In [52]:
prod_to_image = {"Netflix":"netflix.png", 
                 "Apple TV":"apple_tv.png",
                 "CBS": "cbs_all_access.png",
                 "DC": "DCUniverse.png",
                 "Disney": "disney_plus.png",
                 "Hulu": "hulu.png",
                 "Peacock": "peacock.png"}

def create_figure(bar_color="", image_width=75, arrow_size=10):
    
    imgByteArr = io.BytesIO()
    up_arrow = Image.open("arrows/green_arrow.png").resize((arrow_size,arrow_size))
    up_arrow.save(imgByteArr, format='PNG')
    up_arrow = imgByteArr.getvalue()
    
    imgByteArr = io.BytesIO()
    down_arrow = Image.open("arrows/red_arrow.png").resize((arrow_size,arrow_size))
    down_arrow.save(imgByteArr, format='PNG')
    down_arrow = imgByteArr.getvalue()
    
    imgByteArr = io.BytesIO()
    pause_arrow = Image.open("arrows/pause_sign.png").resize((arrow_size,arrow_size))
    pause_arrow.save(imgByteArr, format='PNG')
    pause_arrow = imgByteArr.getvalue()
    
    column1_data = []

    for i, (title, url, flag, production) in enumerate(zip(df["title"], df["poster_url"], df["up_down"], df["production_co"])):
        res = requests.get(url=url)
        
        ### Label 1 Code Starts ###########
        label1 = widgets.HTML(value="<h3>%2.2d</h3>"%(i+1), layout=widgets.Layout(padding="0px"))
        if flag ==1:
            first_row = widgets.VBox([label1, widgets.Image(value=up_arrow, width=20)], layout=widgets.Layout(margin="0px 20px 0px 20px"))
        elif flag == -1:
            first_row = widgets.VBox([label1, widgets.Image(value=down_arrow, width=20)], layout=widgets.Layout(margin="0px 20px 0px 20px"))
        else:
            first_row = widgets.VBox([label1, widgets.Image(value=pause_arrow, width=20)], layout=widgets.Layout(margin="0px 20px 0px 20px"))
            
        ### Label 1 Code Ends ###########
        
        ### Image Code Starts ###############
        imgByteArr = io.BytesIO()
        img = Image.open(io.BytesIO(res.content)).resize((300, 410))
        img.save(imgByteArr, format='PNG')
        img = imgByteArr.getvalue()
        
        image = widgets.Image(value = img, 
                              width=image_width,
                              style={"class":"rounded-circle", "height": "10px"})
        
        ### Image Code Ends ###############
        
        ### Label 2 Code Starts ###############
        label2 = widgets.HTML(value="<h3>%s</h3>"%title, 
                              layout=widgets.Layout(margin="18px"), 
                              style={"color":"blue", "font-weight":"bold"})
        if isinstance(production, str):
            label2.layout = widgets.Layout(margin="18px 18px 0px 18px")
            prod_img = open("production_co/%s"%prod_to_image[production.strip()], mode="rb").read()
            prod_img = widgets.Image(value=prod_img, width=50, layout=widgets.Layout(margin="0px 18px 0px 18px"), )
            label2 = widgets.VBox([label2, prod_img])
            
        ### Label 2 Code Starts ###############
        
        hbox = widgets.HBox([first_row, image, label2])
        hbox.layout.align_items = "center"
        hbox.layout.width="700px"
        column1_data.append(hbox)

    column2_data = []

    max_value = math.ceil(df.occurences.max() / 1e6 + 1)

    for occurence in df.occurences:
            occurence = occurence / 1e6
            label1 = widgets.HTML("<h3 color='blue'>%.2f M</h3>"%occurence, 
                                  layout=widgets.Layout(margin="25px"))
            
            progress = widgets.FloatProgress(value=occurence, 
                                             min=0, max=max_value, 
                                             layout=widgets.Layout(width="400px", height="90px"),
                                             bar_style=bar_color ## success, info, warning, danger
                                            )
            hbox = widgets.HBox([label1, progress])
            hbox.layout.align_items = "center"
            hbox.layout.justify_content = "center"
            hbox.layout.align_content = "center"
            hbox.layout.height="100%"
            hbox.layout.width="100%"
            column2_data.append(hbox)
            
    title = widgets.HBox([widgets.HTML(value="<h4 align='center'><b>Top Television Shows</b><h4>")])
    title.layout.justify_content = "center"
    
    fig = widgets.VBox([widgets.HBox([col1, col2], style={"border":"1px"}) for col1, col2 in zip(column1_data, column2_data)], layout=widgets.Layout(border="2px", margin="1px"))
    return widgets.VBox([title, fig])

In [53]:
create_figure(image_width=75)

In [54]:
create_figure("success", image_width=75)

In [55]:
create_figure("warning", image_width=75)

In [56]:
create_figure("info", image_width=75)

In [57]:
create_figure("danger", image_width=75)

In [ ]:
def create_figure(bar_color="", image_width=75, arrow_size=100):
    
    up_arrow = Image.open("arrows/green_arrow_new_try.png").resize((arrow_size,arrow_size)).convert(mode="RGBA")
    down_arrow = Image.open("arrows/red_arrow_new.png").resize((arrow_size,arrow_size)).convert(mode="RGBA")
    pause_arrow = Image.open("arrows/pause_arrow_new.png").resize((arrow_size,arrow_size)).convert(mode="RGBA")
    
    
    column1_data = []

    for i, (title, url, flag) in enumerate(zip(df["title"], df["poster_url"], df["up_down"])):
        res = requests.get(url=url)
        img = Image.open(io.BytesIO(res.content))
        x, y = img.size
        
        if flag==1:
            img.paste(up_arrow, (0, y-arrow_size), up_arrow)
            
        elif flag==-1:
            img.paste(down_arrow, (0, y-arrow_size), down_arrow)
        else:
            img.paste(pause_arrow, (0, y-arrow_size), pause_arrow)
        
        imgByteArr = io.BytesIO()
        img.save(imgByteArr, format='PNG')
        img = imgByteArr.getvalue()
        
        label1 = widgets.HTML(value="<h3>%2.2d</h3>"%(i+1), layout=widgets.Layout(margin="20px"))
        label2 = widgets.HTML(value="<h3>%s</h3>"%title, layout=widgets.Layout(margin="20px"), style={"color":"blue", "font-weight":"bold"})
        image = widgets.Image(value = img, 
                              width=image_width, height=10, 
                              style={"class":"rounded-circle", "height": "10px"})
        hbox = widgets.HBox([label1, image, label2])
        hbox.layout.align_items = "center"
        hbox.layout.width="600px"
        column1_data.append(hbox)

    column2_data = []

    max_value = math.ceil(df.occurences.max() / 1e6 + 1)

    for occurence in df.occurences:
            occurence = occurence / 1e6
            label1 = widgets.HTML("<h3 color='blue'>%.2f M</h3>"%occurence, 
                                  layout=widgets.Layout(margin="20px"))
            progress = widgets.FloatProgress(value=occurence, 
                                             min=0, max=max_value, 
                                             layout=widgets.Layout(width="400px", height="90px"),
                                             bar_style=bar_color ## success, info, warning, danger
                                            )
            hbox = widgets.HBox([label1, progress])
            hbox.layout.align_items = "center"
            hbox.layout.height="100%"
            hbox.layout.width="100%"
            column2_data.append(hbox)
    title = widgets.HBox([widgets.HTML(value="<h4 align='center'><b>Top Television Shows</b><h4>")])
    title.layout.justify_content = "center"
    
    fig = widgets.VBox([widgets.HBox([col1, col2], style={"border":"1px"}) for col1, col2 in zip(column1_data, column2_data)], layout=widgets.Layout(border="2px", margin="1px"))
    
    return widgets.VBox([title, fig])

In [ ]:
figure = create_figure()
figure

In [ ]:
create_figure("success", image_width=60)

In [ ]:
create_figure("info", image_width=60)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage,AnnotationBbox

arrow_size = 100

up_arrow = Image.open("arrows/green_arrow_new.png").resize((arrow_size,arrow_size)).convert(mode="RGBA")
down_arrow = Image.open("arrows/red_arrow_new.png").resize((arrow_size,arrow_size)).convert(mode="RGBA")
pause_arrow = Image.open("arrows/pause_arrow_new.png").resize((arrow_size,arrow_size)).convert(mode="RGBA")

def offset_image(coord, url, flag, ax):
    res = requests.get(url=url)
    img = Image.open(io.BytesIO(res.content))
    x, y = img.size

    if flag==1:
        img.paste(up_arrow, (0, y-arrow_size), up_arrow)

    elif flag==-1:
        img.paste(down_arrow, (0, y-arrow_size), down_arrow)
    else:
        img.paste(pause_arrow, (0, y-arrow_size), pause_arrow)

    im = OffsetImage(np.asarray(img), zoom=0.35)
    im.image.axes = ax

    ab = AnnotationBbox(im, (coord, 0),  xybox=(0., -80.), frameon=False,
                        xycoords='data',  boxcoords="offset points", pad=0)

    ax.add_artist(ab)

countries = df.title
valuesA = df.occurences

#with plt.style.context("ggplot"):

fig, ax = plt.subplots(figsize=(25,12))

ax.bar(range(len(countries)), valuesA, width=0.9,align="center", color="lime", alpha=0.3)
#ax.barh(range(len(countries)), valuesA, height=0.5,align="center")
ax.set_xticks(range(len(countries)))
ax.set_xticklabels(countries, fontdict={"size":20, "weight":"bold", "color":"dodgerblue"})
ax.set_yticks([])
ax.tick_params(axis='x', which='major', pad=160, rotation=90)

ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

for i, (url, flag) in enumerate(zip(df.poster_url,df.up_down)):
    offset_image(i, url, flag, ax)
    
for i, occ in enumerate(df.occurences.values):
    o = occ / 1e6
    ax.text(i, occ+500, "%.2f M"%o, ha="center", va="bottom", fontsize=20, color="tomato", fontweight="bold")

plt.savefig("fig1.png")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage,AnnotationBbox

def offset_image(coord, url,flag, ax):
    res = requests.get(url=url)
    img = Image.open(io.BytesIO(res.content))
    
    x, y = img.size

    if flag==1:
        img.paste(up_arrow, (0, y-arrow_size), up_arrow)

    elif flag==-1:
        img.paste(down_arrow, (0, y-arrow_size), down_arrow)
    else:
        img.paste(pause_arrow, (0, y-arrow_size), pause_arrow)
        
    im = OffsetImage(np.asarray(img), zoom=0.35)
    im.image.axes = ax

    ab = AnnotationBbox(im, (coord, 0),  xybox=(0., -80.), frameon=False,
                        xycoords='data',  boxcoords="offset points", pad=0)

    ax.add_artist(ab)

countries = df.title
valuesA = df.occurences

#with plt.style.context("ggplot"):

fig, ax = plt.subplots(figsize=(20,13))

ax.bar(range(len(countries)), valuesA, width=0.9,align="center", color="cyan", alpha=0.3)
#ax.barh(range(len(countries)), valuesA, height=0.5,align="center")
ax.set_xticks(range(len(countries)))
ax.set_xticklabels(countries, fontdict={"size":20, "weight":"bold",})
ax.set_yticks([])
ax.tick_params(axis='x', which='major', pad=160, rotation=90)

ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

for i, (url, flag) in enumerate(zip(df.poster_url,df.up_down)):
    offset_image(i, url, flag, ax)
    
for i, occ in enumerate(df.occurences.values):
    o = occ / 1e6
    ax.text(i, occ+500, "%.2f M"%o, ha="center", va="bottom", fontsize=20, fontweight="bold")

plt.tight_layout()
plt.savefig("fig2.png", bbox_inches="tight")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage,AnnotationBbox

def offset_image(coord, url, flag, ax):
    res = requests.get(url=url)
    img = Image.open(io.BytesIO(res.content))
    x, y = img.size

    if flag==1:
        img.paste(up_arrow, (0, y-arrow_size), up_arrow)

    elif flag==-1:
        img.paste(down_arrow, (0, y-arrow_size), down_arrow)
    else:
        img.paste(pause_arrow, (0, y-arrow_size), pause_arrow)
        
    im = OffsetImage(np.asarray(img), zoom=0.35)
    im.image.axes = ax

    ab = AnnotationBbox(im, (0, coord),  xybox=(-70, 0), frameon=False,
                        xycoords='data',  boxcoords="offset points", pad=0)

    ax.add_artist(ab)

countries = df.title
valuesA = df.occurences

#with plt.style.context("ggplot"):

fig, ax = plt.subplots(figsize=(15,32))

#ax.bar(range(len(countries)), valuesA, width=0.9,align="center", color="lime", alpha=0.9)
ax.barh(list(range(len(countries))), list(valuesA)[::-1], height=0.9,align="center", color="dodgerblue", alpha=0.3)
ax.set_yticks(range(len(countries)))
ax.set_yticklabels(countries[::-1], fontdict={"size":20, "weight":"bold", "color":"tomato"})
ax.set_xticks([])
ax.tick_params(axis='y', which='major', pad=140)

ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

for i, (url, flag) in enumerate(zip(df.poster_url[::-1],df.up_down[::-1])):
    offset_image(i, url, flag, ax)
    
for i, occ in enumerate(df.occurences.values[::-1]):
    o = occ / 1e6
    ax.text(occ+500, i, "%.2f M"%o, ha="left", va="center", fontsize=20, color="dodgerblue", fontweight="bold")
    
plt.savefig("fig3.png")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage,AnnotationBbox

def offset_image(coord, url, flag, ax):
    res = requests.get(url=url)
    img = Image.open(io.BytesIO(res.content))
    x, y = img.size

    if flag==1:
        img.paste(up_arrow, (0, y-arrow_size), up_arrow)

    elif flag==-1:
        img.paste(down_arrow, (0, y-arrow_size), down_arrow)
    else:
        img.paste(pause_arrow, (0, y-arrow_size), pause_arrow)
    
    im = OffsetImage(np.asarray(img), zoom=0.45)
    im.image.axes = ax

    ab = AnnotationBbox(im, (0, coord),  xybox=(-70, 0), frameon=False,
                        xycoords='data',  boxcoords="offset points", pad=0)

    ax.add_artist(ab)

countries = df.title
valuesA = df.occurences

#with plt.style.context("ggplot"):

fig, ax = plt.subplots(figsize=(15,32))

#ax.bar(range(len(countries)), valuesA, width=0.9,align="center", color="lime", alpha=0.9)
ax.barh(list(range(len(countries))), list(valuesA)[::-1], height=0.9,align="center", color="lime", alpha=0.3)
ax.set_yticks(range(len(countries)))
ax.set_yticklabels(countries[::-1], fontdict={"size":20, "weight":"bold", "color":"dodgerblue"})
ax.set_xticks([])
ax.tick_params(axis='y', which='major', pad=140)

ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

for i, (url, flag) in enumerate(zip(df.poster_url[::-1],df.up_down[::-1])):
    offset_image(i, url, flag, ax)
    
for i, occ in enumerate(df.occurences.values[::-1]):
    o = occ / 1e6
    ax.text(occ+500, i, "%.2f M"%o, ha="left", va="center", fontsize=20, color="tomato", fontweight="bold")
    
plt.tight_layout()
plt.savefig("fig4.png")    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage,AnnotationBbox

def offset_image(coord, url, flag, ax):
    res = requests.get(url=url)
    img = Image.open(io.BytesIO(res.content))
    x, y = img.size

    if flag==1:
        img.paste(up_arrow, (0, y-arrow_size), up_arrow)

    elif flag==-1:
        img.paste(down_arrow, (0, y-arrow_size), down_arrow)
    else:
        img.paste(pause_arrow, (0, y-arrow_size), pause_arrow)
    
    im = OffsetImage(np.asarray(img), zoom=0.35)
    im.image.axes = ax

    ab = AnnotationBbox(im, (0, coord),  xybox=(-70, 0), frameon=False,
                        xycoords='data',  boxcoords="offset points", pad=0)

    ax.add_artist(ab)

countries = df.title
valuesA = df.occurences

#with plt.style.context("ggplot"):

fig, ax = plt.subplots(figsize=(15,32))

ax.barh(list(range(len(countries))), list(valuesA)[::-1], height=0.9,align="center", color="lime", alpha=0.3)
ax.set_yticks(range(len(countries)))
ax.set_yticklabels(countries[::-1], fontdict={"size":20, "weight":"bold", })#"color":"dodgerblue"})
ax.set_xticks([])
ax.tick_params(axis='y', which='major', pad=140)

ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

for i, (url, flag) in enumerate(zip(df.poster_url[::-1],df.up_down[::-1])):
    offset_image(i, url, flag, ax)
    
for i, occ in enumerate(df.occurences.values[::-1]):
    ax.text(500, i, "{:,}".format(occ), fontsize=20, fontweight="bold")
    

plt.savefig("fig5.png")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage,AnnotationBbox

def offset_image(coord, url, flag, ax):
    res = requests.get(url=url)
    img = Image.open(io.BytesIO(res.content))
    x, y = img.size

    if flag==1:
        img.paste(up_arrow, (0, y-arrow_size), up_arrow)

    elif flag==-1:
        img.paste(down_arrow, (0, y-arrow_size), down_arrow)
    else:
        img.paste(pause_arrow, (0, y-arrow_size), pause_arrow)
    
    im = OffsetImage(np.asarray(img), zoom=0.45)
    im.image.axes = ax

    ab = AnnotationBbox(im, (0, coord),  xybox=(-70, 0), frameon=False,
                        xycoords='data',  boxcoords="offset points", pad=0)

    ax.add_artist(ab)

countries = df.title
valuesA = df.occurences

#with plt.style.context("ggplot"):

fig, ax = plt.subplots(figsize=(15,32))

ax.barh(list(range(len(countries))), list(valuesA)[::-1], height=0.9,align="center", color="lime", alpha=0.3)
ax.set_yticks(range(len(countries)))
ax.set_yticklabels(countries[::-1], fontdict={"size":20, "weight":"bold", "color":"tomato"})
ax.set_xticks([])
ax.tick_params(axis='y', which='major', pad=140)

ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

for i, (url, flag) in enumerate(zip(df.poster_url[::-1],df.up_down[::-1])):
    offset_image(i, url, flag, ax)
    
for i, occ in enumerate(df.occurences.values[::-1]):
    ax.text(500, i, "{:,}".format(occ), fontsize=20, fontweight="bold", color="dodgerblue")
    
plt.tight_layout()
plt.savefig("fig6.png")